In [1]:
import pandas as pd
test_set = pd.read_excel("../input/summerization/Competition CERIST -- Summarization/Dataset 2 Arabic + English (XL_sum)/Arabic/dataset_XL_sum_v1.0_test_ar.xlsx")


test_set['Summary'] = test_set['Summary'].replace(r'\n', '', regex = True)
test_set['Document'] = test_set['Document'].replace(r'\n', '', regex = True)

print(test_set.shape)

(4687, 2)


In [2]:
from datasets import Dataset
test_ds = Dataset.from_pandas(test_set)


In [3]:
!pip install transformers pyarabic
!git clone https://github.com/aub-mind/arabert 

fatal: destination path 'arabert' already exists and is not an empty directory.


In [4]:
from transformers import BertTokenizer, AutoModelForSeq2SeqLM, pipeline
from arabert.preprocess import ArabertPreprocessor

model_name="lilouuch/mbert2mbert-arabic-text-summarization-finetuned-xsum_arabic_abstractive_final_finaln"
preprocessor = ArabertPreprocessor(model_name="")

tokenizer = BertTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipeline = pipeline("text2text-generation",model=model,tokenizer=tokenizer,max_length=512,truncation=True)




The following encoder weights were not tied to the decoder ['bert/pooler']
The following encoder weights were not tied to the decoder ['bert/pooler']


In [5]:
def summerize(example):
    example['generated_text'] =pipeline(example['Document'],
            pad_token_id=tokenizer.eos_token_id,
            num_beams=3,
            repetition_penalty=3.0
            )[0]['generated_text']
    return example

In [ ]:
test_ds = test_ds.map(summerize)

  0%|          | 0/4687 [00:00<?, ?ex/s]